In [10]:
#Grab in the specific helper libraries we would like

#notes, non-filtering by ID, push to csv
import pandas as pd
import numpy as np
import os
from datetime import datetime
import csv



INPUT_DIR = os.getcwd() + "\\..\\data\\input"
OUTPUT_DIR = os.getcwd() + "\\..\\data\\output"

TEST_FILE_NAME = INPUT_DIR + "\\Interactions 01_26_2022 after 10_31_2021.xlsx"


In [20]:
#make sure we 
print("./Documents/GitHub/hydraGAN_code_time_series/all_data.csv")

c:\dev\Gerontechnology\data wrangling\src\..\data\input\Interactions 01_26_2022 after 10_31_2021.xlsx


In [21]:
df = pd.read_excel(TEST_FILE_NAME)
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

In [22]:

d_str = '1/4/2022 9:54'
def conv_t_hard(d_str):
    #print(d_str)
    #date,other = d_str.split('  ')
    date,curr_time = d_str.split(' ')
    #curr_time, ampm = other.split(' ')
    month,day,year = date.split('/')
    month = int(month)
    year = int(year)
    day = int(day)
    hour,minute = curr_time.split(':')
    hour = int(hour)
    minute = int(minute)
    j = 0
    month_secs = 0
    while j <= month:
        if(j == 2):
            month_secs += 2419200
        elif(j == 4 or j == 6 or j == 9 or j == 11):
            month_secs += 2592000
        else:
            month_secs += 2678400
        j += 1
    secs = (year-1970)*31536000 + month_secs + day*24*60*60 + hour*60*60 + minute*60
    return(secs)
out = conv_t_hard(d_str)
print(out)

1645610040


In [23]:
def conv_time(t_string, day):
    #print(t_string)
    time,dmark = t_string.split(' ')
    hour,minutes = time.split(':')
    offset = 0
    if(dmark == 'PM'):
        offset += 12*60*60
    secs = 60*60*int(hour) + 60*int(minutes) + offset +int(day)*24*60*60
    return(secs)
sec = conv_time('5:41 PM', 2)
print(sec)
    

236460


In [24]:
def cal_by_day(day, sec_gap, parc_time, parc_occur,df):
    '''
    we want two dicts: one that holds the last occurance of each participant,
    and one that holds the number of times a gap of size n secs has occured
    '''
    ids = df['participantId_'+str(day)].dropna()
    times = df['Time_'+str(day)].dropna()
    i = 0
    for rows in ids:
        curr_id = ids[i]
        curr_time = times[i]

        if curr_id in parc_time.keys():
            old_time = parc_time.get(curr_id)
            new_time = times[i]
            new_secs = conv_time(new_time, day)
            if(new_secs - old_time >= sec_gap):
                if curr_id in parc_occur:
                    curr_count = parc_occur.get(curr_id)
                    parc_occur.update({curr_id:curr_count+1})
                else:
                    parc_occur.update({curr_id:1})
            parc_time.update({curr_id:new_secs})
        else:
            new_time = times[i]
            new_secs = conv_time(new_time, day)
            parc_time.update({curr_id:new_secs})

        i += 1
    return(parc_time, parc_occur)

In [25]:
print(df["Time_3"].dropna())

KeyError: 'Time_3'

In [129]:
day = 20
sec_gap = 200
end_day = 23
parc_ids = [95,106] #the ids we actually care about, leave empty if we want all
event_ids = []

parc_time = {}
parc_occur = {}

while day <= end_day:
    parc_time, parc_occur = cal_by_day(day, sec_gap, parc_time, parc_occur, df)
    if(len(parc_ids) == 0):
        print("----------------------------")
        print("----------------------------")
        print("For Day:" + str(day))
        print("Times are:")
        print(parc_time)

        print("Occurrences are:")
        print(parc_occur)

        print("----------------------------")
        print("----------------------------")
        
    day += 1
    


In [130]:
for p in parc_ids:
    print("Number of unique occurences:")
    occ = parc_occur.get(p)
    print(p)

Number of unique occurences:
95
Number of unique occurences:
106


In [131]:
df_long = pd.read_csv('./Documents/GitHub/hydraGAN_code_time_series/all_long.csv')

In [132]:
def break_by_day(d_str,curr_day):
    month,day,dontcare = d_str.split('/')
    if(int(day)!= curr_day):
        return(1)
    return(0)
def get_current_day(d_str):
    #print(d_str)
    month,day,dontcare = d_str.split('/')
    return(int(day))

def cal_by_day_single(day, sec_gap, parc_time, parc_occur,df,el_ids,curr_day):
    '''
    we want two dicts: one that holds the last occurance of each participant,
    and one that holds the number of times a gap of size n secs has occured
    '''
    ids = df['participantId'].dropna()
    times = df['timestamp_local'].dropna()
    i = 0
    for rows in ids:
        curr_id = ids[i]
        curr_time = times[i]

        if curr_id in parc_time.keys():
            old_time = parc_time.get(curr_id)
            new_time = times[i]
            new_secs = conv_t_hard(new_time)
            if(new_secs - old_time >= sec_gap):
                if curr_id in parc_occur:
                    if(len(el_ids) == 0 or df['elementId'][i] in el_ids):
                        curr_count = parc_occur.get(curr_id)
                        parc_occur.update({curr_id:curr_count+1})
                else:
                    parc_occur.update({curr_id:1})
            parc_time.update({curr_id:new_secs})
        else:
            new_time = times[i]
            new_secs = conv_t_hard(new_time)
            parc_time.update({curr_id:new_secs})

        i += 1
    return(parc_time, parc_occur)

In [133]:
def filter_df_to_correct_days(df, first_day, end_day):
    #print(df.loc[0, 'timestamp_local'])
    
    new_df = pd.DataFrame()
    #new_df.columns = df.columns
    f_day = df.loc[0, 'timestamp_local']
    current_date = get_current_day(f_day)
    curr_d_index = 1
    i = 0
    while i < df.shape[0]:
        new_date = get_current_day(df.loc[i, 'timestamp_local'])
        if(new_date != current_date):
            curr_d_index += 1
            current_date = new_date 
        #print(curr_d_index, first_day)
        if(int(curr_d_index) >= first_day and int(curr_d_index) <= end_day):
            #new_df.append(df.loc[i])
            new_df = pd.concat([new_df, df.loc[i]],axis=1)
        i += 1
    return(new_df.T)
    
new_df = filter_df_to_correct_days(df_long, 3,4)
#print(df_long.shape)
print(new_df.head(3))

    interactionId participantId elementId  timestamp_local  \
225           226            48       263  10/18/2021 9:13   
226           227            48         8  10/18/2021 9:13   
227           228            48         3  10/18/2021 9:13   

    timestamp_utcoffset interaction token type  
225                -700         tap   NaN  NaN  
226                -700         tap   NaN  NaN  
227                -700     present   NaN  NaN  


In [134]:
#put access calculations here:
    

In [148]:
df_long = pd.read_csv('./Documents/GitHub/hydraGAN_code_time_series/all_long.csv') #Put the path to the csv you care about here
start_day = 1 # day to start
sec_gap = 20 #how many seconds between interactions before we care about it
end_day = 2 #day to end
parc_ids = [] #the ids we actually care about, leave empty if we want all
elem_ids = [] #the elements we want, leave blank if we want all

parc_time = {}
parc_occur = {}

'''
filter by day starting day

dict to csv (locally)
'''
#curr_day = get_current_day(df_long['timestamp_local'][0])

#print("We are starting on day:", curr_day)
df_long = filter_df_to_correct_days(df_long,start_day,end_day)


parc_time, parc_occur = cal_by_day_single(day, sec_gap, parc_time, parc_occur, df_long,elem_ids, curr_day)
if(len(parc_ids) == 0):
    print("----------------------------")
    print("----------------------------")
    #print("For Day:" + str(day))
    print("Times are:")
    #print(parc_time)

    print("Occurrences are:")
    print(parc_occur)

    print("----------------------------")
    print("----------------------------")
       

----------------------------
----------------------------
Times are:
Occurrences are:
{48: 14, 75: 1}
----------------------------
----------------------------


In [147]:
with open('./output.csv', 'w') as csvfile:
    csvwriter = csv.writer(csvfile) 
    csvwriter.writerow(['Participant', 'Count'])
    for key,value in parc_occur.items():
        print(key,value)
        csvwriter.writerow([key, value])

48 14
75 1


In [5]:
def get_interaction_counts(df, elementIDs):
    """Given a dataframe which contains participant interactions,
    returns a count of interactions for the given elementID for each participant.

    Args:
        df (Pandas Dataframe): Created earlier from read_excel
        elementID (int): The elementID number to be aggregated.
    """
    query_string = ""
    for i in range(len(elementIDs)):
        query_string += 'elementId == {}'.format(elementIDs[i])
        if((i+1) < len(elementIDs)):
            query_string += " or "
    
    count = df.query(query_string)
    return count.groupby(['participantId', 'elementId']).size()
    


In [7]:
def create_variable(participants_dict, interactions_df, variable_name, elementIDs, variable_func):
    # Each interaction is a pair ([participantID, elementID], count)
    element_count_list = get_interaction_counts(interactions_df, elementIDs).iteritems()

    for element_count in element_count_list:
        participant_id = element_count[0][0]

        if participant_id in participants_dict:
            participants_dict[participant_id][variable_name] = variable_func(element_count[1])
        else:
            participants_dict[participant_id] = dict()
            participants_dict[participant_id][variable_name] = variable_func(element_count[1])
    
    for participant_id in participants_dict:
        if variable_name not in participants_dict[participant_id]:
            participants_dict[participant_id][variable_name] = 0
    
    return participants_dict



In [34]:
def participant_dict_to_csv(participants, outfile_name="output.csv"):
    interactions_df = pd.DataFrame.from_dict(participants, orient='index')
    interactions_df.reset_index(inplace=True)
    interactions_df.rename({'index':'participantId'}, axis='columns', inplace=True)
    interactions_df.to_csv(OUTPUT_DIR + "\\{}".format(outfile_name), index=False)

In [35]:
def get_variable_calculations(interactions_filename, data_date_range=7):
    participants = dict()

    interactions = pd.read_excel(interactions_filename)

    participants = create_variable(participants, interactions, "CalenderUse", [9], lambda x: x)

    participant_dict_to_csv(participants)



get_variable_calculations(TEST_FILE_NAME)